In [ ]:
### SBB.py program

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import HiveContext, SQLContext,SparkSession
from pyspark.sql import functions as F
from datetime import date,timedelta
from datetime import datetime
sc = SparkContext(appName = "Combined_Table")
sql_context = HiveContext(sc)
spark = SparkSession(sc)
spark = (SparkSession.builder.enableHiveSupport().getOrCreate())


In [ ]:
spark 
import pandas as pd
import os
import numpy as np
import datetime
import time
import glob
import os
from numpy import nan
from pyspark.sql import functions as F
from pyspark.sql.functions import format_string

pd.set_option('display.width', 150)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
exec(open('/home/thural2/THURAL2/thural2/record.py').read())


In [ ]:
rec_date

In [ ]:
####################### FIRST PROCESS THE GROSS VOLUME SEGMENTATION METRICS###########################################
cred_fil=spark.sql("select * from anp_cabbtdct1_final.psa_elig_amsb")
cred_sbb_filter=cred_fil.toPandas()

## change psa_month to month name
cred_sbb_filter=cred_sbb_filter.astype({"psa_month": int})
import calendar
cred_sbb_filter['psa_month'] = cred_sbb_filter['psa_month'].apply(lambda x: calendar.month_name[x]).str.lower()
cred_sbb_filter.head(10)

In [ ]:
amsb_cred=cred_sbb_filter.groupby(by=['pr_acf2','employee_name','psa_month',
                                      'metric_name','am_cost_center',
                                      'am_cost_center_name','am_cost_center_full_name',
                                      'sm_cost_center','sm_cost_center_name',
                                      'record_date'],as_index=False).aggregate({
                    'elig_ind_final': 'sum'
                     })
amsb_cred.rename(columns = {"elig_ind_final":'count_segment'}, inplace = True)

##transpose data
amsb_cred1=amsb_cred.pivot_table(index=['pr_acf2','employee_name','metric_name',
                                        'am_cost_center','am_cost_center_name',
                                        'am_cost_center_full_name',
                                        'sm_cost_center','sm_cost_center_name',
                                        'record_date'], columns=['psa_month'],
                     values='count_segment', aggfunc='first').reset_index().fillna(0)

In [ ]:
amsb_cred1.head(10)

In [ ]:
########## CREATE ALL OF THE MONTHS
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in amsb_cred1]

amsb_cred1.reset_index(drop=True, inplace=True)
amsb_cred2= pd.concat([net_a[complementary], amsb_cred1], axis=1, join='outer')

##delete the extra filler row
amsb_cred2= amsb_cred2.drop(labels='a', axis=0)
##convert to float
amsb_cred2 = amsb_cred2.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})

amsb_cred2.rename(columns = {"pr_acf2":'acf2_id'}, inplace = True)
amsb_cred2['scorecard_filter']='SBB'
amsb_cred2['level']='AMSB'

## order relevant vars for amsb_cred3
amsb_cred3=amsb_cred2[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
## also group by to get the total widgets
amsb_cred4=amsb_cred3.groupby(by=['acf2_id','employee_name','am_cost_center',
                                  'am_cost_center_name','am_cost_center_full_name',
                                  'sm_cost_center','sm_cost_center_name',
                                  'scorecard_filter','level','record_date'],as_index=False).aggregate({
                        'november': 'sum',
                        'december': 'sum',
                        'january': 'sum',
                        'february': 'sum',
                        'march': 'sum',
                        'april': 'sum',
                        'may': 'sum',
                        'june': 'sum',
                        'july': 'sum',
                        'august': 'sum',
                        'september': 'sum',
                        'october': 'sum'
                     })

amsb_cred4['metric_name']='Total SBB Credit Widgets'

## order relevant vars for amsb_cred4
amsb_cred5=amsb_cred4[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

In [ ]:
### APPEND TOTAL TO THE OTHER SUB-CATEGORIES
amsb_cred6=pd.concat([amsb_cred5,amsb_cred3],axis=0)


In [ ]:
## NOW create QTD and YTD variables
newvarfunc(amsb_cred6)

## replace 0 with NaN so median only counts non-zeros
amsb_cred6=amsb_cred6.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
cred_amsb_median=amsb_cred6.groupby(['metric_name','sm_cost_center'], as_index=False)['q1'].median()
##rename median
cred_amsb_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
cred_amsb_all3=pd.merge(amsb_cred6,cred_amsb_median,on=['metric_name','sm_cost_center'],
                   how="left")

## Q2
cred_amsb_median1=amsb_cred6.groupby(['metric_name','sm_cost_center'], as_index=False)['q2'].median()
##rename median
cred_amsb_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
cred_amsb_all4=pd.merge(cred_amsb_all3,cred_amsb_median1,on=['metric_name','sm_cost_center'],
                   how="left")

## Q3
cred_amsb_median2=amsb_cred6.groupby(['metric_name','sm_cost_center'], as_index=False)['q3'].median()
##rename median
cred_amsb_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
cred_amsb_all5=pd.merge(cred_amsb_all4,cred_amsb_median2,on=['metric_name','sm_cost_center'],
                   how="left")

## Q4
cred_amsb_median3=amsb_cred6.groupby(['metric_name','sm_cost_center'], as_index=False)['q4'].median()
##rename median
cred_amsb_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
cred_amsb_all6=pd.merge(cred_amsb_all5,cred_amsb_median3,on=['metric_name','sm_cost_center'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
cred_amsb_ytdmedian=amsb_cred6.groupby(['metric_name','sm_cost_center'], as_index=False)['ytd'].median()
##rename median
cred_amsb_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
cred_amsb_all7=pd.merge(cred_amsb_all6,cred_amsb_ytdmedian,on=['metric_name','sm_cost_center'],
                   how="left")

cred_amsb_all8=cred_amsb_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]
cred_amsb_all8.info()

In [ ]:
##############################  GROUP BY AM (SBB FILTER)##############################
cred_am=amgrpfunc(cred_amsb_all8)

##include the LEVEL variable
cred_am['level']='AM'
cred_am['scorecard_filter']='SBB'
cred_am['acf2_id']=''
cred_am['employee_name']=cred_am['am_cost_center_full_name']

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(cred_am)

## replace 0 with NaN so median only counts non-zeros
cred_am=cred_am.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
cred_am_median=cred_am.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
cred_am_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all3=pd.merge(cred_am,cred_am_median,on=['metric_name'],
                   how="left")

## Q2
cred_am_median1=cred_am.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
cred_am_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all4=pd.merge(cred_am_all3,cred_am_median1,on=['metric_name'],
                   how="left")

## Q3
cred_am_median2=cred_am.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
cred_am_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all5=pd.merge(cred_am_all4,cred_am_median2,on=['metric_name'],
                   how="left")

## Q4
cred_am_median3=cred_am.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
cred_am_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all6=pd.merge(cred_am_all5,cred_am_median3,on=['metric_name'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
cred_am_ytdmedian=cred_am.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
cred_am_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
cred_am_all7=pd.merge(cred_am_all6,cred_am_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
cred_am_all8=cred_am_all7[['metric_name','level','acf2_id','employee_name',
               'november','december','january','february','march','april','may','june','july',
               'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
##############################  GROUP BY SM (SBB FILTER)##############################
cred_sm=smgrpfunc(cred_am_all8)

##include the LEVEL variable
cred_sm['level']='SM'
cred_sm['acf2_id']=''
cred_sm['scorecard_filter']='SBB'
cred_sm['employee_name']=cred_sm['sm_cost_center_name']
cred_sm['am_cost_center']=''
cred_sm['am_cost_center_name']=''
cred_sm['am_cost_center_full_name']=''

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(cred_sm)

## replace 0 with NaN so median only counts non-zeros
cred_sm=cred_sm.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
cred_sm_median=cred_sm.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
cred_sm_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all3=pd.merge(cred_sm,cred_sm_median,on=['metric_name'],
                   how="left")

## Q2
cred_sm_median1=cred_sm.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
cred_sm_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all4=pd.merge(cred_sm_all3,cred_sm_median1,on=['metric_name'],
                   how="left")

## Q3
cred_sm_median2=cred_sm.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
cred_sm_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all5=pd.merge(cred_sm_all4,cred_sm_median2,on=['metric_name'],
                   how="left")

## Q4
cred_sm_median3=cred_sm.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
cred_sm_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all6=pd.merge(cred_sm_all5,cred_sm_median3,on=['metric_name'],
                   how="left")

## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
cred_sm_ytdmedian=cred_sm.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
cred_sm_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
cred_sm_all7=pd.merge(cred_sm_all6,cred_sm_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
cred_sm_all8=cred_sm_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
######################################################################################
############################# NOW PROCESS PSA METRICS ################################
## spark sql
df_fil=spark.sql("select * from anp_cabbtdct1_final.SCORECARD_SBB_FILTER")
mif_sbb_filter=df_fil.toPandas()
mif_sbb_filter.head(10)

## rename the numeric months to full month name
mif_sbb_filter.rename(columns = {'month11':'november', 'month12': 'december',
                       'month1':'january','month2': 'february', 'month3':'march', 'month4': 'april',
                       'month5':'may', 'month6': 'june','month7':'july', 'month8': 'august',
                       'month9':'september', 'month10': 'october'}, inplace = True)

##convert to float
mif_sbb_filter = mif_sbb_filter.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})

## filter out VOLUME
df_mif_volume=mif_sbb_filter[(mif_sbb_filter.bal_typ=='BM') & (mif_sbb_filter.sub_typ=='A')]

## filter out WIDGETS
df_mif_widgets=mif_sbb_filter[(mif_sbb_filter.bal_typ=='BM') & (mif_sbb_filter.sub_typ=='B')]

############################# GROSS VOLUME ###########################################
## SBB TAB: TOTAL SBB GROSS VOLUME
sbb_gross_filter=df_mif_volume[df_mif_volume.prod_nm.isin(gross_all_tuple)]
sbb_gross_filter=sbb_gross_filter.assign(metric_name='Total SBB Gross Volume')
sbb_gross_filter=filterfunc(sbb_gross_filter)

## SBB GROSS LOC VOLUME
gross_loc_filter=df_mif_volume[df_mif_volume.prod_nm.isin(gross_loc_tuple)]
gross_loc_filter=gross_loc_filter.assign(metric_name='SBB Gross LOC Volume')
gross_loc_filter=filterfunc(gross_loc_filter)

## SBB GROSS LON VOLUME
gross_lon_filter=df_mif_volume[df_mif_volume.prod_nm.isin(gross_lon_tuple)]
gross_lon_filter=gross_lon_filter.assign(metric_name='SBB Gross LON Volume')
gross_lon_filter=filterfunc(gross_lon_filter)

## SBB GROSS PROFESSIONAL STUDENT LOC VOLUME
gross_prof_filter=df_mif_volume[df_mif_volume.prod_nm.isin(gross_student_loc_tuple)]
gross_prof_filter=gross_prof_filter.assign(metric_name='Gross Professional Student LOC')
gross_prof_filter=filterfunc(gross_prof_filter)

### APPEND ALL THE DATAFRAME FILTERS
gross_all=pd.concat([sbb_gross_filter,gross_loc_filter,gross_lon_filter,gross_prof_filter],axis=0)

########################## CREDIT CARDS ##########################################

## SBB TAB: TOTAL CREDIT CARDS
sbb_cc_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_cc_tuple)]
sbb_cc_filter=sbb_cc_filter.assign(metric_name='Total Credit Cards')
sbb_cc_filter=filterfunc(sbb_cc_filter)

## SBB TAB: BUSINESS VISA
sbb_bv_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(business_visa_cc_tuple)]
sbb_bv_filter=sbb_bv_filter.assign(metric_name='Credit Cards - Business VISA')
sbb_bv_filter=filterfunc(sbb_bv_filter)

## SBB TAB: BUSINESS VISA
sbb_btv_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(business_travel_tuple)]
sbb_btv_filter=sbb_btv_filter.assign(metric_name='Credit Cards - Business Travel VISA')
sbb_btv_filter=filterfunc(sbb_btv_filter)

## SBB TAB: AEROPLAN BUSINESS
sbb_aero_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(business_aero_tuple)]
sbb_aero_filter=sbb_aero_filter.assign(metric_name='Credit Cards - Aeroplan Business')
sbb_aero_filter=filterfunc(sbb_aero_filter)

## SBB TAB: BUSINESS SELECT NO FEE
sbb_nofee_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(bus_nofee_tuple)]
sbb_nofee_filter=sbb_nofee_filter.assign(metric_name='Credit Cards - Business Select No Fee')
sbb_nofee_filter=filterfunc(sbb_nofee_filter)

## SBB TAB: BUSINESS SELECT FEE
sbb_fee_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(bus_fee_tuple)]
sbb_fee_filter=sbb_fee_filter.assign(metric_name='Credit Cards - Business Select Fee')
sbb_fee_filter=filterfunc(sbb_fee_filter)

### APPEND ALL THE DATAFRAME FILTERS
cc_all=pd.concat([sbb_cc_filter,sbb_bv_filter,sbb_btv_filter,sbb_aero_filter,
                  sbb_nofee_filter,sbb_fee_filter],axis=0)

########################## MERCHANT SOLUTIONS ##########################################

## SBB TAB: TOTAL MERCHANT SOLUTIONS
merchant_all=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_mer_tuple)]
merchant_all=merchant_all.assign(metric_name='Total Merchant Solutions')
merchant_all=filterfunc(merchant_all)

################################ CMS ###############################################

## SBB TAB: TOTAL CMS
cms_all=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_cms_tuple)]
cms_all=cms_all.assign(metric_name='Total Cash Management Services (including RDC)')
cms_all=filterfunc(cms_all)

################################ COMMERCIAL VOLUME ###################################

## SBB TAB: TOTAL COMMERCIAL VOLUME
sbb_com_filter=df_mif_volume[df_mif_volume.prod_nm.isin(total_com_tuple)]
sbb_com_filter=sbb_com_filter.assign(metric_name='ALL Total Commercial Volume')
sbb_com_filter=filterfunc(sbb_com_filter)

## SBB TAB: COMMERCIAL DEPOSIT VOLUME
sbb_dep_filter=df_mif_volume[df_mif_volume.prod_nm.isin(com_deposit_tuple)]
sbb_dep_filter=sbb_dep_filter.assign(metric_name='All Commercial Deposit Volume')
sbb_dep_filter=filterfunc(sbb_dep_filter)

## SBB TAB: COMMERCIAL LOAN VOLUME
sbb_loan_filter=df_mif_volume[df_mif_volume.prod_nm.isin(com_loan_tuple)]
sbb_loan_filter=sbb_loan_filter.assign(metric_name='All Commercial Loan Volume')
sbb_loan_filter=filterfunc(sbb_loan_filter)

### APPEND ALL THE DATAFRAME FILTERS
com_all=pd.concat([sbb_com_filter,sbb_dep_filter,sbb_loan_filter],axis=0)

################################ NEW ACCOUNT OPENINGS ###############################

## SBB TAB: TOTAL ACCOUNT OPENINGS
sbb_open_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_new_tuple)]
sbb_open_filter=sbb_open_filter.assign(metric_name='Total New SBB Chequing Accounts')
sbb_open_filter=filterfunc(sbb_open_filter)

################################ TD SECURITIES DIRECT TRADE ###########################

## SBB TAB: TOTAL TD SECURITIES DIRECT TRADE
sbb_trade_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_trade_tuple)]
sbb_trade_filter=sbb_trade_filter.assign(metric_name='TD Securities Direct Trade')
sbb_trade_filter=filterfunc(sbb_trade_filter)

################################ BCP WIDGETS ##########################################

## SBB TAB: TOTAL BCP WIDGETS
sbb_bcp_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_bcp_tuple)]
sbb_bcp_filter=sbb_bcp_filter.assign(metric_name='Business Credit Protection(BCP) Widgets')
sbb_bcp_filter=filterfunc(sbb_bcp_filter)

############################# MUR MORTGAGE VOLUME######################################

## SBB TAB: MUR MORTGAGE VOLUME
sbb_mur_filter=df_mif_volume[df_mif_volume.prod_nm.isin(total_mur_tuple)]
sbb_mur_filter=sbb_mur_filter.assign(metric_name='Total MUR Mortgage Volume')
sbb_mur_filter=filterfunc(sbb_mur_filter)

################################ WEALTH REFERRAL WIDGETS #############################

## SBB TAB: WEALTH REFERRAL WIDGETS
sbb_wealth_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_wealth_tuple)]
sbb_wealth_filter=sbb_wealth_filter.assign(metric_name='Total Wealth Referrals')
sbb_wealth_filter=filterfunc(sbb_wealth_filter)

########################################### BODP ############################################

## SBB TAB: BODP
sbb_bodp_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_bodp_tuple)]
sbb_bodp_filter=sbb_bodp_filter.assign(metric_name='Total BODP')
sbb_bodp_filter=filterfunc(sbb_bodp_filter)

############################ CONCATENATE ALL DATAFRAMES ###############################

### APPEND ALL THE DATAFRAME FILTERS (exclude commercial - will process separately for AMSB)
psa_all=pd.concat([gross_all,cc_all,merchant_all,cms_all, 
                   sbb_open_filter,sbb_trade_filter,sbb_bcp_filter,
                   sbb_mur_filter,sbb_wealth_filter,sbb_bodp_filter],axis=0)

### APPEND ALL THE DATAFRAME FILTERS (includes commercial for AM/SM rollup)
psa_all99=pd.concat([gross_all,cc_all,merchant_all,cms_all, com_all,
                   sbb_open_filter,sbb_trade_filter,sbb_bcp_filter,
                   sbb_mur_filter,sbb_wealth_filter,sbb_bodp_filter],axis=0)

psa_all.metric_name.unique()

In [ ]:
############################# PROCESS COMMERCIAL VOLUME FOR AMSB ################################
## read in retail file
## spark sql
df_com_fil=spark.sql("select * from anp_cabbtdct1_final.SCORECARD_RETAIL_FILTER where prod_nm in {}".format(tuple(total_com_tuple)))
mif_com_filter=df_com_fil.toPandas()

## rename the numeric months to full month name
mif_com_filter.rename(columns = {'month11':'november', 'month12': 'december',
                       'month1':'january','month2': 'february', 'month3':'march', 'month4': 'april',
                       'month5':'may', 'month6': 'june','month7':'july', 'month8': 'august',
                       'month9':'september', 'month10': 'october'}, inplace = True)

##convert to float
mif_com_filter = mif_com_filter.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})

## filter out VOLUME
df_com_volume=mif_com_filter[(mif_com_filter.bal_typ=='BM') & (mif_com_filter.sub_typ=='A')]


In [ ]:
######################## CATEGORIZE between loan and deposit and total#############################################

com_filter=df_com_volume[df_com_volume.prod_nm.isin(total_com_tuple)]
com_filter=com_filter.assign(metric_name='ALL Total Commercial Volume')
com_filter=filterfunc(com_filter)

## RETAIL TAB: COMMERCIAL DEPOSIT VOLUME
com_dep_filter=df_com_volume[df_com_volume.prod_nm.isin(com_deposit_tuple)]
com_dep_filter=com_dep_filter.assign(metric_name='All Commercial Deposit Volume')
com_dep_filter=filterfunc(com_dep_filter)

## RETAIL TAB: COMMERCIAL LOAN VOLUME
com_loan_filter=df_com_volume[df_com_volume.prod_nm.isin(com_loan_tuple)]
com_loan_filter=com_loan_filter.assign(metric_name='All Commercial Loan Volume')
com_loan_filter=filterfunc(com_loan_filter)

### APPEND ALL THE DATAFRAME FILTERS
com_all_ret=pd.concat([com_filter,com_dep_filter,com_loan_filter],axis=0)


In [ ]:
## read in latest buddy branch table
df_bud_max=spark.sql("select max(record_date) as max_rec from anp_cabbtdct1_final.buddy_branch")
df1 = spark.table('anp_cabbtdct1_final.BUDDY_BRANCH')
df2=df1.withColumn("branch_0000",format_string("%04d","branch"))
df3 = df2.join(df_bud_max, how='inner', on= F.col('record_date')==F.col('max_rec'))
buddy_branch=df3.toPandas()

## merge COM dataframe to buddy branch file
df_retail_buddy=pd.merge(com_all_ret,buddy_branch[['branch_0000','logon']],
                   how="left",left_on=['branch_no'],right_on=['branch_0000'],indicator=True)

## filter both records
df_retail_buddy1=df_retail_buddy[(df_retail_buddy['_merge'] == 'both')]

## drop 'employee_name' --this is blank because it's a retail employee
df_retail_buddy1=df_retail_buddy1.drop(columns=['employee_name','acf2_id','_merge'])


In [ ]:
df_retail_buddy1.info()

In [ ]:
############# MERGE THE RETAIL FILTER TO HRM_SBB_ALIGNMENT TO GET 'EMPLOYEE_NAME' #############################
sbb_hr = spark.table('anp_cabbtdct1_final.hrm_sbb_alignment')
sbb_hr1=sbb_hr.toPandas()

## select most recent record date
sbb_hr1=sbb_hr1[(sbb_hr1.record_date==rec_date)]

## merge retail filter to hrm file
retail_fil_amsb=pd.merge(df_retail_buddy1,sbb_hr1[['acf2_id','employee_name']],
                   how="left",left_on=['logon'],right_on=['acf2_id'],indicator=True)

### filter only AMSBs; there could also be AMs who did sales but won't be picked up now; pick up in rollup
retail_fil_amsb1=retail_fil_amsb[(retail_fil_amsb['_merge'] == 'both')]


In [ ]:
retail_fil_amsb1.info()

In [ ]:
########################## CONCATENATE THE SBB AND RETAIL FILTERS FOR COMMERCIAL ##############################

## SBB filter: keep relevant vars
com_all1=filterfunc(com_all)

## RETAIL filter: keep relevant vars and rename logon to acf2_id
df_retail_buddy2=retail_fil_amsb[['metric_name','key_typ','logon','year','prod_nm','bal_typ',
                 'sub_typ','branch_no','wk_tot', 'november','december',
                 'january','february','march','april','may','june','july','august',
                 'september','october','period_end_date','employee_name',
                 'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                 'sm_cost_center','sm_cost_center_name','record_date']]

##rename logon to acf2_id
df_retail_buddy2.rename(columns={"logon":'acf2_id'}, inplace = True)
df_retail_buddy2=filterfunc(df_retail_buddy2)

## concatenate amsb (sbb) and amsb (retail) for commercial
com_all_amsb=pd.concat([com_all1,df_retail_buddy2],axis=0)

## concatenate the commercial dataframe to the rest of the metrics (excluding commercial)
psa_amsb=pd.concat([psa_all,com_all_amsb],axis=0)


In [ ]:
###############################  GROUP BY AMSB FOR ALL METRICS (based on SM region) ##########################################
psa_amsb=grpfunc(psa_amsb)
                        
##include the LEVEL variable
psa_amsb['level']='AMSB'
psa_amsb['scorecard_filter']='SBB'

##CREATE THE AMSB QUARTER AND YTD VARIABLES
newvarfunc(psa_amsb)

## replace 0 with NaN so median only counts non-zeros
psa_amsb=psa_amsb.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
psa_amsb_median=psa_amsb.groupby(['metric_name','sm_cost_center'], as_index=False)['q1'].median()
##rename median
psa_amsb_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
psa_amsb_all3=pd.merge(psa_amsb,psa_amsb_median,on=['metric_name','sm_cost_center'],
                   how="left")

## Q2
psa_amsb_median1=psa_amsb.groupby(['metric_name','sm_cost_center'], as_index=False)['q2'].median()
##rename median
psa_amsb_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
psa_amsb_all4=pd.merge(psa_amsb_all3,psa_amsb_median1,on=['metric_name','sm_cost_center'],
                   how="left")

## Q3
psa_amsb_median2=psa_amsb.groupby(['metric_name','sm_cost_center'], as_index=False)['q3'].median()
##rename median
psa_amsb_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
psa_amsb_all5=pd.merge(psa_amsb_all4,psa_amsb_median2,on=['metric_name','sm_cost_center'],
                   how="left")

## Q4
psa_amsb_median3=psa_amsb.groupby(['metric_name','sm_cost_center'], as_index=False)['q4'].median()
##rename median
psa_amsb_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
psa_amsb_all6=pd.merge(psa_amsb_all5,psa_amsb_median3,on=['metric_name','sm_cost_center'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
psa_amsb_ytdmedian=psa_amsb_all6.groupby(['metric_name','sm_cost_center'], as_index=False)['ytd'].median()
##rename median
psa_amsb_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
psa_amsb_all7=pd.merge(psa_amsb_all6,psa_amsb_ytdmedian,on=['metric_name','sm_cost_center'],
                   how="left")

## keep relevant vars
psa_amsb_all8=psa_amsb_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
##############################  GROUP BY AM (SBB FILTER)##############################
psa_am=amgrpfunc(psa_all99)

##include the LEVEL variable
psa_am['level']='AM'
psa_am['scorecard_filter']='SBB'
psa_am['acf2_id']=''
psa_am['employee_name']=psa_am['am_cost_center_full_name']

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(psa_am)

## replace 0 with NaN so median only counts non-zeros
psa_am=psa_am.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
psa_am_median=psa_am.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
psa_am_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
psa_am_all3=pd.merge(psa_am,psa_am_median,on=['metric_name'],
                   how="left")

## Q2
psa_am_median1=psa_am.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
psa_am_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
psa_am_all4=pd.merge(psa_am_all3,psa_am_median1,on=['metric_name'],
                   how="left")

## Q3
psa_am_median2=psa_am.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
psa_am_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
psa_am_all5=pd.merge(psa_am_all4,psa_am_median2,on=['metric_name'],
                   how="left")

## Q4
psa_am_median3=psa_am.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
psa_am_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
psa_am_all6=pd.merge(psa_am_all5,psa_am_median3,on=['metric_name'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
psa_am_ytdmedian=psa_am.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
psa_am_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
psa_am_all7=pd.merge(psa_am_all6,psa_am_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
psa_am_all8=psa_am_all7[['metric_name','level','acf2_id','employee_name',
               'november','december','january','february','march','april','may','june','july',
               'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
##############################  GROUP BY SM (SBB FILTER)##############################
psa_sm=smgrpfunc(psa_am)

##include the LEVEL variable
psa_sm['level']='SM'
psa_sm['acf2_id']=''
psa_sm['scorecard_filter']='SBB'
psa_sm['employee_name']=psa_sm['sm_cost_center_name']
psa_sm['am_cost_center']=''
psa_sm['am_cost_center_name']=''
psa_sm['am_cost_center_full_name']=''

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(psa_sm)

## replace 0 with NaN so median only counts non-zeros
psa_sm=psa_sm.replace(0.0, np.nan)


## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
psa_sm_median=psa_sm.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
psa_sm_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
psa_sm_all3=pd.merge(psa_sm,psa_sm_median,on=['metric_name'],
                   how="left")

## Q2
psa_sm_median1=psa_sm.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
psa_sm_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
psa_sm_all4=pd.merge(psa_sm_all3,psa_sm_median1,on=['metric_name'],
                   how="left")

## Q3
psa_sm_median2=psa_sm.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
psa_sm_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
psa_sm_all5=pd.merge(psa_sm_all4,psa_sm_median2,on=['metric_name'],
                   how="left")

## Q4
psa_sm_median3=psa_sm.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
psa_sm_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
psa_sm_all6=pd.merge(psa_sm_all5,psa_sm_median3,on=['metric_name'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
psa_sm_ytdmedian=psa_sm.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
psa_sm_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
psa_sm_all7=pd.merge(psa_sm_all6,psa_sm_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
psa_sm_all8=psa_sm_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


### the 3 dataframes (AMSB, AM, SM) have aggregated month1-month12 and q1-q4,ytd

### APPEND ALL THE DATAFRAME FILTERS including the segmentation metrics
df_sbb_all=pd.concat([psa_amsb_all8,psa_am_all8,psa_sm_all8,
                      cred_amsb_all8,cred_am_all8,cred_sm_all8],axis=0)



In [ ]:
df_sbb_all.head(10)

In [ ]:
df_sbb_all.metric_name.unique()

In [ ]:
#spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
## overwrite
#df_sbb_all.write.mode("overwrite").saveAsTable("anp_cabbtdct1_final.SBB_SC_METRICS_SBB")

In [ ]:
## Convert pandas dataframe to Pyspark dataframe and save in table
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.createDataFrame(df_sbb_all).write.mode("append").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_METRICS_SBB") 

In [ ]:
## END OF PROGRAM